In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install chess
!pip install python-chess ipywidgets


In [ ]:
import numpy as np
import pandas as pd
import chess
import random
import tensorflow as tf
from tensorflow import keras

In [ ]:
model=keras.models.load_model("/content/drive/MyDrive/chess_comp/model_2_1.h5")


def Preprocess_Data(board1):

    # Setting up the piece values
    piece_values = {
        'P': 11, 'p': -11, 'N': 44, 'n': -44, 'B': 33, 'b': -33, 'R': 55,
        'r': -55, 'Q': 99, 'q': -99, 'K': 110, 'k': -110
    }

    # Mapping piece symbols to bitboard layers
    piece_to_layer = {
        'P': 0, 'p': 1, 'N': 2, 'n': 3, 'B': 4, 'b': 5, 'R': 6, 'r': 7, 'Q': 8,
        'q': 9, 'K': 10, 'k': 11
    }

    # Create an empty bitboard of shape (16, 8, 8) using numpy
    en_passant_layer = 12
    white_values_layer = 13
    black_values_layer = 14
    valid_move_positions_layer = 15

    bitboard = np.zeros((16, 8, 8))

    board=board1
    if board.ep_square is not None:
        rank, file = divmod(board.ep_square, 8)
        bitboard[en_passant_layer][rank][file] = 1 if board.turn == chess.WHITE else -1
    computed_moves = {}

    for piece_type, piece_value in piece_values.items():
        layer = piece_to_layer[piece_type]

        if piece_type not in computed_moves:
            valid_moves = set()
            empty_board_moves = set()
            for square in chess.SQUARES:
                piece = board.piece_at(square)
                if piece and piece.symbol() == piece_type:
                    # Generate moves for both colors irrespective of turn
                    board.turn = chess.WHITE
                    moves_white = [move for move in board.legal_moves if move.from_square == square]
                    board.turn = chess.BLACK
                    moves_black = [move for move in board.legal_moves if move.from_square == square]
                    valid_moves.update([move.to_square for move in moves_white])
                    valid_moves.update([move.to_square for move in moves_black])

                    empty_board = chess.Board()
                    empty_board.clear()
                    empty_board.set_piece_at(square, piece)
                    # Generate moves for both colors irrespective of turn on the empty board
                    empty_board.turn = chess.WHITE
                    empty_moves_white = [move for move in empty_board.generate_legal_moves(from_mask=chess.BB_SQUARES[square])]
                    empty_board.turn = chess.BLACK
                    empty_moves_black = [move for move in empty_board.generate_legal_moves(from_mask=chess.BB_SQUARES[square])]
                    empty_board_moves.update([move.to_square for move in empty_moves_white])
                    empty_board_moves.update([move.to_square for move in empty_moves_black])

            computed_moves[piece_type] = (valid_moves, empty_board_moves)

        valid_moves, empty_board_moves = computed_moves[piece_type]

        for square in chess.SQUARES:
            rank, file = divmod(square, 8)
            piece = board.piece_at(square)
            if piece:
                if piece.color == chess.WHITE:
                    bitboard[white_values_layer][rank][file] = piece_values[piece.symbol()]
                else:
                    bitboard[black_values_layer][rank][file] = piece_values[piece.symbol()]
            if piece and piece.symbol() == piece_type:
                bitboard[layer][rank][file] = piece_value
            elif square in valid_moves:
                bitboard[layer][rank][file] = piece_value / 11
            elif square in empty_board_moves:
                bitboard[layer][rank][file] = piece_value / 88
        board = board1
        for move in board.legal_moves:
            rank, file = divmod(move.from_square, 8)
            if board.turn == chess.WHITE:
              bitboard[valid_move_positions_layer][rank][file] = 1
            else:
              bitboard[valid_move_positions_layer][rank][file] = -1

    transposed_array = np.transpose(bitboard, (1, 2, 0))
    reshaped_array = np.expand_dims(transposed_array, axis=0)

    return reshaped_array




**Evaluation Model**

In [ ]:

def evaluate_board(board):
    data=Preprocess_Data(board)
    score=model.predict(data)
    print(score)
    return score

**Best Move**

In [ ]:
def best_move_using_simple_evaluation(board):
    white_to_play = board.turn
    best_score = -999999 if white_to_play else 999999
    best_move=None
    for move in board.legal_moves:
        board.push(move)
        score = evaluate_board(board)
        board.pop()
        if score > best_score and white_to_play:
            best_score = score
            best_move = move
        elif score < best_score and not white_to_play:
            best_score = score
            best_move = move
    return best_move

**MiniMax Algorithm**


In [ ]:
def minimax(board, depth, alpha, beta, white_to_play):
    if depth == 0:
        return evaluate_board(board)
    if white_to_play:
        best_score = -999999
        best_move = None
        for move in board.legal_moves:
            board.push(move)
            score= minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            if score > best_score:
                best_score = score
                best_move = move
            alpha = max(alpha, best_score)
            if alpha >= beta:
                break
        return best_score
    else:
        best_score = 999999
        best_move = None
        for move in board.legal_moves:
            board.push(move)
            score = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            if score < best_score:
                best_score = score
                best_move = move
            beta = min(beta, best_score)
            if alpha >= beta:
                break
        return best_score




def best_move_using_minimax(board, depth):
    white_to_play = board.turn
    best_score = -9999999 if white_to_play else 9999999
    best_move = random_move(board)
    for move in board.legal_moves:
        board.push(move)
        score = minimax(board, depth - 1,-1000000,1000000, not white_to_play)
        board.pop()
        if score > best_score and white_to_play:
            best_score = score
            best_move = move
        elif score < best_score and not white_to_play:
            best_score = score
            best_move = move
    return best_move

In [ ]:
import chess
import chess.svg
import ipywidgets as widgets
from IPython.display import display, clear_output, SVG

board = chess.Board()
source_square = None

symbols = {
    'P': '♙', 'N': '♘', 'B': '♗', 'R': '♖', 'Q': '♕', 'K': '♔',
    'p': '♟', 'n': '♞', 'b': '♝', 'r': '♜', 'q': '♛', 'k': '♚'
}

def create_button(square):
    piece = board.piece_at(square)
    symbol = symbols[piece.symbol()] if piece else ' '
    background_color = 'white' if (chess.square_rank(square) + chess.square_file(square)) % 2 == 0 else 'black'
    button_color = 'black' if piece and piece.color == chess.BLACK else 'white'

    button = widgets.Button(description=symbol, layout=widgets.Layout(width='60px', height='60px'))
    button.style.button_color = background_color
    button.style.font_weight = 'bold'
    button.square = square
    button.on_click(on_square_clicked)
    return button


def on_square_clicked(button):
    global source_square
    if source_square is None:
        source_square = button.square
    else:
        move = chess.Move(source_square, button.square)
        if move in board.legal_moves:
            board.push(move)
            # ai_move = best_move_using_minimax(board, 2)  # Adjust depth as needed
            ai_move = best_move_using_simple_evaluation(board)
            if ai_move:
                board.push(ai_move)
        source_square = None
        update_board()

def update_board():
    clear_output(wait=True)

    # Create the interactive board
    rows = []
    for rank in reversed(range(8)):
        buttons = [create_button(chess.square(file, rank)) for file in range(8)]
        rows.append(widgets.HBox(buttons))

    interactive_board = widgets.VBox(rows)

    # Get the SVG representation of the board
    board_svg = chess.svg.board(board=board)

    # Convert SVG data to string format suitable for HTML embedding
    svg_str = '<svg width="400" height="400">' + board_svg + '</svg>'

    # Embed SVG data in an HTML widget
    svg_widget = widgets.HTML(value=svg_str)

    # Display both side by side
    display(widgets.HBox([interactive_board, svg_widget]))

update_board()